In [19]:
import pandas as pd
import numpy as np 
import tensorflow as tf
from matplotlib import pyplot as plt
import collections
import random
pixel_depth = 255

def prep_data(csv):
    data = pd.read_csv(csv)
    labels = None
    #One-hot encoding for labels
    if csv == 'train.csv':
        labels = (np.arange(10) == (data['label'])[:,None]).astype(np.float32)
    #Pull individual pixel values from dataframe as flat array
    image = ["%s%s" %("pixel",pixel_no) for pixel_no in range(0,784)]
    data = np.array(data[image])
    #Reshape so the data is back to original dimensions
    data = data.astype('float32').reshape(-1, 28, 28, 1)
    #Rescale pixel values from a range of 0-255 to 0-1
    data = (data - np.min(data))/np.ptp(data)
    return data, labels

def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation, :]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

train_data, train_labels = prep_data('train.csv')
train_data, train_labels = randomize(train_data, train_labels)

In [24]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

#import test data 
test_data = mnist.test.images
test_labels = mnist.test.labels

num_steps = 1000
batch_size = 100

graph = tf.Graph()

with graph.as_default():
    tf_train_data = tf.placeholder(tf.float32, [None, 28, 28, 1])
    tf_train_labels = tf.placeholder(tf.float32, [None, 10])
    
    #Format for weights [filter size, other filter size dimension, input channels, output channels]
    #Input channels is one because the images are grayscale, as opposed to rgb which would be 3
    w1 = tf.Variable(tf.truncated_normal([5,5,1,4], stddev=0.1))
    b1 = tf.Variable(tf.ones([4])/10)
    w2 = tf.Variable(tf.truncated_normal([5,5,4,8], stddev=0.1))
    b2 = tf.Variable(tf.ones([8])/10)
    w3 = tf.Variable(tf.truncated_normal([4,4,8,12], stddev=0.1))
    b3 = tf.Variable(tf.ones([12])/10)
    
    w4 = tf.Variable(tf.truncated_normal([7*7*12, 200], stddev=0.1))
    b4 = tf.Variable(tf.ones([200])/10)
    w5 = tf.Variable(tf.truncated_normal([200, 10], stddev=0.1))
    b5 = tf.Variable(tf.ones([10])/10)
    
    stride = 1
    y1 = tf.nn.relu(tf.nn.conv2d(tf_train_data, w1, strides=[1, stride, stride, 1], padding='SAME') + b1)
    stride = 2 
    y2 = tf.nn.relu(tf.nn.conv2d(y1, w2, strides=[1, stride, stride, 1], padding='SAME') + b2)
    stride = 2 
    y3 = tf.nn.relu(tf.nn.conv2d(y2, w3, strides=[1, stride, stride, 1], padding='SAME') + b3)
    
    reshaped_data = tf.reshape(y3, shape=[-1, 7*7*12])
    
    y4 = tf.nn.relu(tf.matmul(reshaped_data, w4) + b4)
    logits = tf.matmul(y4, w5) + b5
    y = tf.nn.softmax(logits)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=tf_train_labels))
    
    correct_pred = tf.equal(tf.argmax(y,1), tf.argmax(tf_train_labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    lr = 0.003
    
    optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    print('Initialized variables.')
    for step in range(num_steps + 1):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        batch_data = train_data[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
            
        _ = sess.run(optimizer, feed_dict={tf_train_data: batch_data, tf_train_labels: batch_labels})
            
        if step % 10 == 0:
            print('Training step: ' + str(step) + ' Accuracy: ' + str(sess.run(accuracy, feed_dict={tf_train_data: batch_data, tf_train_labels: batch_labels})) 
                  + ' Loss: ' + str(sess.run(loss, feed_dict={tf_train_data: batch_data, tf_train_labels: batch_labels})))
        
    #Need to figure out how to correctly display test data results in an informative way on tensorboard
    print('Test accuracy: ' + str(sess.run(accuracy, feed_dict={tf_train_data: test_data, tf_train_labels: test_labels})))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Initialized variables.
Training step: 0 Accuracy: 0.19 Loss: 2.1959968
Training step: 10 Accuracy: 0.64 Loss: 1.2668141
Training step: 20 Accuracy: 0.85 Loss: 0.4501469


KeyboardInterrupt: 